# 프로젝트 : 멋진 작사가 만들기

### 라이브러리 버전 확인

In [1]:
import glob
import tensorflow as tf

print(tf.__version__)

2.6.0


### Step 1. 데이터 다운로드


### Step 2. 데이터 읽어오기

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path) 

raw_corpus = [] 


for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() 
        raw_corpus.extend(raw) 

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


### Step 3. 데이터 정제

In [3]:
import re
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


# 이 문장이 어떻게 필터링되는지 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛰기
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split(' ')) <=15 :
        corpus.append(preprocessed_sentence)
    
# 정제된 결과를 30개 확인
corpus[:30]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> she broke your throne , and she cut your hair <end>',
 '<start> and from your lips she drew the hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah you say i took the name in vain <end>',
 '<start> i don t even know the name <end>',
 '<start> th

preprocessed_sentence = preprocess_sentence(sentence)

    if len(preprocessed_sentence.split(' ')) <=15 :
    
        corpus.append(preprocessed_sentence)
        
--> 

토큰화 했을때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외시키고 

corpus에 append 하기.

In [5]:
len(corpus)

156013

In [6]:
# print(len(corpus[0].split(' ')))

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer
    # 우리는 이미 문장을 정제했으니 filters가 필요없다
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾼다
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
   
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f9748527490>


In [8]:
print(tensor[:3, :10]) # 행 , 렬

[[   2   50    4   95  303   62   53    9  946 6263]
 [   2   15 2967  871    5    8   11 5739    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]


In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [10]:
src_input = tensor[:, :-1]  

tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


In [11]:
print(len(src_input))
print(len(tgt_input))

156013
156013


### Step 4. 평가 데이터셋 분리

In [12]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input, 
                                                          test_size = 0.2, 
                                                          shuffle=True, 
                                                          random_state=10)

In [13]:
# print(enc_train)
# print(enc_val)
# print(dec_train)
# print(dec_val)

print(len(enc_train))
print(len(enc_val))
print(len(dec_train))
print(len(dec_val))

124810
31203
124810
31203


### Step 5. 인공지능 만들기

In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE


VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만든다
# 데이터셋에 대해서는 아래 문서를 참고
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
embedding_size = 256 
hidden_size = 1024 
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) 

In [16]:
for src_sample, tgt_sample in dataset.take(1): break


model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.50610785e-04,  2.28238685e-04, -1.25838385e-04, ...,
          9.10183226e-05,  9.41522812e-05, -2.96658545e-04],
        [-2.45328672e-04,  2.34896026e-04, -2.62794696e-04, ...,
         -8.40862558e-05,  2.67433468e-04, -2.36708875e-04],
        [-2.37111817e-04,  2.86232244e-04, -1.62003314e-04, ...,
         -2.66933261e-04,  3.48057074e-04,  9.48211600e-05],
        ...,
        [-2.50884459e-05, -8.12940300e-04, -7.24632002e-04, ...,
         -4.14517883e-04,  9.09898139e-04,  1.03275140e-03],
        [ 1.89090370e-05, -1.17992004e-03, -8.29560391e-04, ...,
         -4.60609124e-04,  1.16841902e-03,  1.09895505e-03],
        [ 7.41773329e-05, -1.50798017e-03, -9.21876170e-04, ...,
         -4.99588263e-04,  1.38547854e-03,  1.16246182e-03]],

       [[-2.50610785e-04,  2.28238685e-04, -1.25838385e-04, ...,
          9.10183226e-05,  9.41522812e-05, -2.96658545e-04],
        [-2.35149113e-04,  4.49794024e-04, -2

In [17]:
# 모델의 구조를 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [18]:
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수
    from_logits=True,
    reduction='none' 
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정

In [19]:
model.fit(dataset, epochs=30)
#Loss
# tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') 

Epoch 1/30
609/609 [==============================] - 96s 150ms/step - loss: 3.4127
Epoch 2/30
609/609 [==============================] - 99s 162ms/step - loss: 2.9593
Epoch 3/30
609/609 [==============================] - 103s 168ms/step - loss: 2.7931
Epoch 4/30
609/609 [==============================] - 106s 173ms/step - loss: 2.6636
Epoch 5/30
609/609 [==============================] - 106s 173ms/step - loss: 2.5517
Epoch 6/30
609/609 [==============================] - 106s 173ms/step - loss: 2.4505
Epoch 7/30
609/609 [==============================] - 106s 173ms/step - loss: 2.3572
Epoch 8/30
609/609 [==============================] - 106s 173ms/step - loss: 2.2703
Epoch 9/30
609/609 [==============================] - 106s 173ms/step - loss: 2.1895
Epoch 10/30
609/609 [==============================] - 105s 173ms/step - loss: 2.1135
Epoch 11/30
609/609 [==============================] - 105s 173ms/step - loss: 2.0412
Epoch 12/30
609/609 [==============================] - 105s 173ms

In [20]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): 
    test_input = tokenizer.texts_to_sequences([init_sentence]) 
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

# 훈련한 모델로 다양한 가사 만들어보기-!

##### 시작문장으로 i love 를 넣어 문장생성 함수 실행

In [53]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20) 

'<start> i love you , i m not gonna crack <end> '

##### 시작문장으로 i 를 넣어 문장생성 함수 실행

In [54]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len=20) 

'<start> i m gonna be a little selfish <end> '

##### 시작문장으로 i am 을 넣어 문장생성 함수 실행

In [42]:
generate_text(model, tokenizer, init_sentence="<start> i am", max_len=100) 

'<start> i am not throwing away my shot <end> '

##### 시작문장으로 i like 를 넣어 문장생성 함수 실행

In [43]:
generate_text(model, tokenizer, init_sentence="<start> i like", max_len=20) 

'<start> i like it when you tro it pon me <end> '

##### 시작문장으로 my 를 넣어 문장생성 함수 실행

In [44]:
generate_text(model, tokenizer, init_sentence="<start> my", max_len=40) 

'<start> my name is who ? <end> '

##### 시작문장으로 you 를 넣어 문장생성 함수 실행

In [49]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=30) 

'<start> you know i m bad , i m bad you know it <end> '

##### 시작문장으로 my love 를 넣어 문장생성 함수 실행

In [50]:
generate_text(model, tokenizer, init_sentence="<start> my love", max_len=30)

'<start> my love for you is eternal one <end> '

# 회고

- 이번 프로젝트에서 **어려웠던 점,**

이전 fund에서 NLP에대해 처음접해보았는데, 개념부터 너무 어려웠었다.

model fit을 하는데 시간이 너무 오래걸려서 힘들었다.


- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.

함수와 코드를 하나하나보면서 이해했어야했는데, 그러지 못한 상태로 마무리를 하게되어 아쉽다.

자연어 처리를 위해서 정규표현식을 했는데, 정규표현식을 좀 더 공부해야겠다.

옵티마이저로 Adam을 사용했는데, 다른 옵티마이저를 사용하면 어떻게 될지 궁금해졌다.

하이퍼파라미터의 값을 바꾸며 학습을 시켜보았을땐 어떨지 도전해봐야겠다.



- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.

특수문자 제거, 토크나이저 생성, 토큰화 했을때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외시키기, 
단어장크기 12,000장 이상, 평가데이터셋 총 데이터의 20%, 

10 Epoch 안에 val_loss값 2.2 줄여보기

Epoch 10/30
609/609 [==============================] - 105s 173ms/step - loss: 2.1135

텍스트 제너레이션 결과로 생성된 문장이 해석가능한 문장인지 살펴보기



- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정.느낌

이전 노드의 내용을 그대로 참고해 학습부분에서 스스로 고민해본것이 부족했다.


- **자기 다짐**

관심있는 분야와 그렇지 않은 분야를 정해놓고 어느 한쪽에 집중하는건 맞지만 

학습에서도 소홀히 하면 결국 나에게 좋을건 없다는걸 다시한번 더 깨닳았다.

Embedding 개념을 다시 한번 살펴보고 이번 프로젝트와 19번 fund의 내용을 익힐 수 있게 노력해야겠다.